In [1]:
class CFG:
    target = "gesture"
    folds = 5
    seed = 42
    folder_path = 'csvfile/'
    except_cols = ['row_id']
    importance_threshold=1e-4
    target_gestures = [
        'Above ear - pull hair',
        'Cheek - pinch skin',
        'Eyebrow - pull hair',
        'Eyelash - pull hair',
        'Forehead - pull hairline',
        'Forehead - scratch',
        'Neck - pinch skin',
        'Neck - scratch',
        ]
    non_target_gestures = [
        'Write name on leg',
        'Wave hello',
        'Glasses on/off',
        'Text on phone',
        'Write name in air',
        'Feel around in tray and pull out an object',
        'Scratch knee/leg skin',
        'Pull air toward your face',
        'Drink from bottle/cup',
        'Pinch knee/leg skin'
        ]
    all_gestures = target_gestures + non_target_gestures

In [2]:
import pandas as pd
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import cv2
import itertools
import random
from scipy.stats import zscore, skew
from scipy.stats import ks_2samp
from sklearn.base import BaseEstimator, ClassifierMixin,TransformerMixin, RegressorMixin,clone
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score, train_test_split
from sklearn.feature_selection import SelectFromModel
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from scipy.stats import norm
from sklearn.tree import DecisionTreeRegressor
import itertools
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
train = pl.read_csv(CFG.folder_path + 'train.csv').join(pl.read_csv(CFG.folder_path + 'train_demographics.csv'), on="subject", how="left")
test = pl.read_csv(CFG.folder_path + 'test.csv').join(pl.read_csv(CFG.folder_path + 'test_demographics.csv'), on="subject", how="left")
tof_cols_ = [f'tof_{s}_v{p}' for s in range(1, 6) for p in range(64)]

In [4]:
def calculate_contest_metric(y_true, y_pred, target_gestures):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    target_gestures = set(target_gestures)
    
    all_labels = sorted(list(target_gestures | {'non_target'}))

    y_true_binary = np.where(np.isin(y_true, list(target_gestures)), 'target', 'non_target')
    y_pred_binary = np.where(np.isin(y_pred, list(target_gestures)), 'target', 'non_target')
    
    binary_f1 = f1_score(y_true_binary, y_pred_binary, pos_label='target', average='binary')

    y_true_macro = np.where(np.isin(y_true, list(target_gestures)), y_true, 'non_target')
    y_pred_macro = np.where(np.isin(y_pred, list(target_gestures)), y_pred, 'non_target')

    macro_f1 = f1_score(y_true_macro, y_pred_macro, labels=all_labels, average='macro')

    final_score = (binary_f1 + macro_f1) / 2

    return {
        'binary_f1': binary_f1,
        'macro_f1': macro_f1,
        'final_score': final_score
    }

# 2. Make Pipelines

In [5]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Polars 데이터프레임 X에서 self.columns를 선택
        selected_X = X.select(self.columns)
        return selected_X

In [6]:
class ToFDataset(Dataset):
    """
    null 값을 원천적으로 방어하도록 수정된 최종 데이터셋 클래스
    """
    def __init__(self, df, tof_cols, labels=None):
        # --- [핵심 수정] ---
        # Polars -> Numpy로 변환하기 전에, 모든 null 값을 -1.0으로 채웁니다.
        self.tof_data = (
            df.select(tof_cols)
            .fill_null(-1.0)
            .to_numpy()
            .astype(np.float32)
        )
        # -------------------
        
        # 만약 이 단계에서도 NaN이 있다면, 원본 데이터 자체에 Inf 등의 이상값이 있다는 의미입니다.
        if np.isnan(self.tof_data).any():
            print("🚨 CRITICAL: NaN detected in data even after fill_null. Check source for Inf values.")
            # Inf 값을 -1로 대체
            self.tof_data = np.nan_to_num(self.tof_data, nan=-1.0, posinf=-1.0, neginf=-1.0)

        self.labels = labels

    def __len__(self):
        return len(self.tof_data)

    def __getitem__(self, idx):
        tof_vector = self.tof_data[idx]
        tof_image = tof_vector.reshape(5, 8, 8)
        
        # 이 단계에서는 -1만 존재하므로, 정규화 로직은 그대로 유지합니다.
        valid_data_mask = (tof_image != -1.0)
        normalized_image = np.where(valid_data_mask, tof_image / 254.0, 0)
        final_image = np.where(valid_data_mask, normalized_image, -1.0)
        
        tof_tensor = torch.tensor(final_image, dtype=torch.float32)

        if self.labels is not None:
            return tof_tensor, self.labels[idx]
        else:
            return tof_tensor

# CNN 모델 클래스
class ToF_CNN_Extractor(nn.Module):
    def __init__(self, num_classes, feature_dim=128):
        super().__init__()
        
        self.feature_extractor = nn.Sequential(
            # Input: (N, 5, 8, 8)
            nn.Conv2d(in_channels=5, out_channels=32, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32), 
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(), # -> (N, 64 * 2 * 2) = (N, 256)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(64 * 2 * 2, feature_dim), # <-- 64*2*2 에서 128*2*2 (즉, 512)로 수정
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(0.5),
            nn.Linear(feature_dim, num_classes)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(64 * 2 * 2, feature_dim),
            nn.LeakyReLU(negative_slope=0.01), # ReLU -> LeakyReLU
            nn.Dropout(0.5),
            nn.Linear(feature_dim, num_classes)
        )

    def forward(self, x):
        features = self.feature_extractor(x)
        output = self.classifier(features)
        return output

# -----------------------------------------------------
# CNNFeatureGenerator 클래스 수정
# -----------------------------------------------------
class CNNFeatureGenerator(BaseEstimator, TransformerMixin):
    def __init__(self, num_classes, feature_dim=256, epochs=5, batch_size=64, lr=1e-4, verbose=True):
        self.num_classes = num_classes
        self.feature_dim = feature_dim
        self.epochs = epochs
        self.batch_size = batch_size
        self.lr = lr
        self.verbose = verbose
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        # ToF_CNN_Extractor 모델은 이전에 수정한 '더 커진' 버전을 사용합니다.
        self.model = ToF_CNN_Extractor(self.num_classes, self.feature_dim).to(self.device)
        
        self.tof_cols_ = [f'tof_{s}_v{p}' for s in range(1, 6) for p in range(64)]
        
        # --- [핵심 수정 부분] ---
        # Flatten 후의 실제 피처 개수인 512에 맞춰 피처 이름을 생성합니다.
        self.flattened_dim_ = 64 * 2 * 2  # 512
        self.feature_names_out_ = [f'cnn_feat_{i}' for i in range(self.flattened_dim_)]
        # ------------------------

    def fit(self, X, y=None):
        # fit 메서드는 이전과 동일합니다.
        if self.verbose:
            print(f"CNN Feature Generator: Starting training on {self.device} for {self.epochs} epochs...")
        # (이하 생략)
        
        train_dataset = ToFDataset(X, self.tof_cols_, labels=y)
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()
        
        self.model.train()
        for epoch in range(self.epochs):
            total_loss = 0
            for inputs, labels in train_loader:
                # labels.to() 호출 시 dtype=torch.long을 추가
                inputs, labels = inputs.to(self.device), labels.to(self.device, dtype=torch.long) 
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels) # 이제 정상 작동
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                optimizer.step()
                total_loss += loss.item()

            if self.verbose:
                if len(train_loader) > 0:
                    avg_loss = total_loss / len(train_loader)
                    print(f"  Epoch {epoch+1}/{self.epochs}, Loss: {avg_loss:.6f}")
                else:
                    print(f"  Epoch {epoch+1}/{self.epochs}, train_loader is empty.")
        return self

    def transform(self, X):
        # transform 메서드는 이전과 동일합니다.
        if self.verbose:
            print("CNN Feature Generator: Extracting features...")
        self.model.eval()
        transform_dataset = ToFDataset(X, self.tof_cols_, labels=None)
        transform_loader = DataLoader(transform_dataset, batch_size=self.batch_size * 2, shuffle=False)
        all_features = []
        with torch.no_grad():
            for inputs in transform_loader:
                inputs = inputs.to(self.device)
                features = self.model.feature_extractor(inputs)
                all_features.append(features.cpu().numpy())
        cnn_features_np = np.concatenate(all_features, axis=0)
        
        # 이제 스키마(512)와 데이터(512)의 차원이 일치합니다.
        return pl.DataFrame(cnn_features_np, schema=self.get_feature_names_out())

    def get_feature_names_out(self, input_features=None):
        return self.feature_names_out_

# 3. Train Model

In [7]:
class HybridFullStackingClassifier(BaseEstimator, ClassifierMixin):
    # __init__ 메소드는 변경 없음 (이전과 동일)
    def __init__(self, pipelines, model_defs, meta_model,
                 n_splits=CFG.folds, random_state=CFG.seed, verbose=True,
                 stratify_feature=None, n_bins=10):
        self.pipelines = pipelines
        self.model_defs = model_defs
        self.meta_model = meta_model
        self.n_splits = n_splits
        self.random_state = random_state
        self.verbose = verbose
        self.stratify_feature = stratify_feature
        self.n_bins = n_bins
        self.fitted_models_ = []
        self.label_encoder_ = LabelEncoder()
        self.classes_ = None
        self.feature_names_in_ = None

    def fit(self, X, y):
        # ... (상단 부분은 이전 Polars 버전과 동일) ...
        if not isinstance(X, pl.DataFrame):
            raise TypeError("Input X must be a Polars DataFrame.")
        if not isinstance(y, pl.Series):
            raise TypeError("Input y must be a Polars Series.")

        self.feature_names_in_ = X.columns
        y_encoded = self.label_encoder_.fit_transform(y.to_numpy())
        self.classes_ = self.label_encoder_.classes_

        n_samples = X.height
        n_classes_out = len(self.classes_) if len(self.classes_) > 2 else 1
        total_base_models = len(self.pipelines) * len(self.model_defs)
        meta_features = np.zeros((n_samples, total_base_models * n_classes_out))

        stratify_values = y_encoded
        if self.stratify_feature and self.stratify_feature in X.columns:
            binner = KBinsDiscretizer(n_bins=self.n_bins, encode='ordinal', strategy='quantile')
            stratify_values = binner.fit_transform(X.select(self.stratify_feature).to_pandas()).astype(int).flatten()

        kf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)
        
        base_model_counter = 0
        for pi, pipeline_def in enumerate(self.pipelines):
            for mi, (model_cls, model_params) in enumerate(self.model_defs):
                if self.verbose:
                    print(f"\nTraining Pipeline {pi+1} + Model {model_cls.__name__}")

                oof_preds = np.zeros((n_samples, n_classes_out))
                fold_specific_models = []

                for fold, (train_idx, val_idx) in enumerate(kf.split(np.zeros(n_samples), stratify_values)):
                    if self.verbose: print(f"  Fold {fold+1}/{self.n_splits}...")
                    
                    X_train, X_val = X.slice(train_idx[0], len(train_idx)), X.slice(val_idx[0], len(val_idx))
                    y_train_fold, y_val_fold = y_encoded[train_idx], y_encoded[val_idx]
                    
                    pipeline = clone(pipeline_def).fit(X_train, y_train_fold)
                    
                    X_train_trans = pipeline.transform(X_train)
                    X_val_trans = pipeline.transform(X_val)

                    cat_cols = X_train_trans.select(cs.string() | cs.categorical()).columns
                    
                    model_params_copy = model_params.copy()
                    model_name = model_cls.__name__.lower()
                    
                    if "catboost" in model_name:
                        model_params_copy["cat_features"] = cat_cols
                    elif "xgb" in model_name:
                        model_params_copy["enable_categorical"] = True
                    
                    model = model_cls(**model_params_copy)
                    
                    # [PANDAS-CONVERSION] 모델 학습 직전에 Pandas로 변환
                    X_train_pd = X_train_trans.to_pandas()
                    X_val_pd = X_val_trans.to_pandas()

                    if "lgbm" in model_name:
                         # LightGBM은 category 타입을 직접 지원
                         X_train_pd[cat_cols] = X_train_pd[cat_cols].astype("category")
                         X_val_pd[cat_cols] = X_val_pd[cat_cols].astype("category")
                         model.fit(X_train_pd, y_train_fold) 
                    else:
                         model.fit(X_train_pd, y_train_fold)
                    
                    proba_preds = model.predict_proba(X_val_pd) # 변환된 Pandas 데이터로 예측
                    
                    if n_classes_out == 1:
                        oof_preds[val_idx] = proba_preds[:, 1].reshape(-1, 1)
                    else:
                        oof_preds[val_idx] = proba_preds
                    
                    fold_specific_models.append((pipeline, model))
                
                start_col, end_col = base_model_counter, base_model_counter + n_classes_out
                meta_features[:, start_col:end_col] = oof_preds
                base_model_counter += n_classes_out
                
                self.fitted_models_.append(fold_specific_models)

        self.meta_model.fit(meta_features, y_encoded)
        return self

    def predict_proba(self, X):
        if not isinstance(X, pl.DataFrame):
            raise TypeError("Input X must be a Polars DataFrame.")
        
        n_samples = X.height
        n_classes_out = len(self.classes_) if len(self.classes_) > 2 else 1
        total_base_models = len(self.fitted_models_)
        meta_X_test = np.zeros((n_samples, total_base_models * n_classes_out))
        
        current_meta_col_idx = 0
        for fold_models in self.fitted_models_:
            fold_preds = []
            for pipeline, model in fold_models:
                X_trans = pipeline.transform(X)

                # [PANDAS-CONVERSION] 모델 예측 직전에 Pandas로 변환
                X_trans_pd = X_trans.to_pandas()
                
                # LGBM을 위한 범주형 타입 변환
                cat_cols = X_trans.select(cs.string() | cs.categorical()).columns
                if cat_cols:
                    X_trans_pd[cat_cols] = X_trans_pd[cat_cols].astype("category")

                fold_preds.append(model.predict_proba(X_trans_pd))

            averaged_preds = np.mean(np.array(fold_preds), axis=0)
            
            start_col, end_col = current_meta_col_idx, current_meta_col_idx + n_classes_out
            if n_classes_out == 1:
                meta_X_test[:, start_col:end_col] = averaged_preds[:, 1].reshape(-1, 1)
            else:
                meta_X_test[:, start_col:end_col] = averaged_preds
            current_meta_col_idx += n_classes_out
            
        return self.meta_model.predict_proba(meta_X_test)
    
    # predict와 score 메소드는 변경 없음
    def predict(self, X):
        final_proba = self.predict_proba(X)
        if len(self.classes_) == 2:
            predictions_encoded = (final_proba[:, 1] >= 0.5).astype(int)
        else:
            predictions_encoded = np.argmax(final_proba, axis=1)
        return self.label_encoder_.inverse_transform(predictions_encoded)

    def score(self, X, y):
        y_pred = self.predict(X)
        # y가 polars series일 수 있으므로 numpy로 변환
        y_true = y.to_numpy() if isinstance(y, pl.Series) else np.array(y)
        return calculate_contest_metric(y_true, y_pred, CFG.target_gestures)

In [ ]:
def build_column_selector_pipeline(columns):
    return Pipeline([
        ('column_selector', ColumnSelector(columns))
    ])

def build_cnn_feature_pipeline(num_classes, 
                               feature_dim=64, 
                               epochs=50, 
                               batch_size=128, 
                               lr=1e-3, 
                               verbose=True):
    return Pipeline([
        ('cnn_features', CNNFeatureGenerator(
            num_classes=num_classes, 
            feature_dim=feature_dim,
            epochs=epochs,
            batch_size=batch_size,
            lr=lr,
            verbose=verbose
        ))
    ])

In [9]:
train_X_full = train.drop(['row_id', CFG.target])
train_y_full = train[CFG.target]
train_X_full_pd = train_X_full.to_pandas()
train_y_full_pd = train_y_full.to_pandas()

X_train_pd, X_val_pd, y_train_pd, y_val_pd = train_test_split(
    train_X_full_pd, train_y_full_pd,
    test_size=0.2,
    random_state=CFG.seed,
    stratify=train_y_full_pd
)
X_train_20 = pl.from_pandas(X_train_pd)
X_val_20 = pl.from_pandas(X_val_pd)
y_train_20 = pl.from_pandas(y_train_pd)
y_val_20 = pl.from_pandas(y_val_pd)

In [10]:
pipeline1 = build_column_selector_pipeline(columns=tof_cols_)
pipeline2 = build_cnn_feature_pipeline(num_classes=len(CFG.all_gestures))
pipelines = [pipeline1,pipeline2] 

In [11]:
model_defs = [
    (CatBoostClassifier, {
        'iterations': 2000,
        'learning_rate': 0.05,
        'depth': 8,
        'loss_function': 'MultiClass',
        'l2_leaf_reg': 3,
        'random_seed': CFG.seed,
        'eval_metric': 'MultiClass',
        'early_stopping_rounds': 200,
        'verbose': 0,
        'task_type': 'GPU',
        'bootstrap_type': 'Poisson',
        'grow_policy': 'Depthwise',
        'subsample': 0.8
    }),
    (XGBClassifier, {
        'max_depth': 8,
        'colsample_bytree': 0.5,
        'subsample': 0.9,
        'n_estimators': 2000,
        'learning_rate': 0.05,
        'gamma': 0.01,
        'reg_alpha': 2.0,
        'reg_lambda': 1.5,
        'max_delta_step': 2,
        'eval_metric': 'mlogloss',
        'random_state': CFG.seed,
        'tree_method': 'hist',
        'device': 'cuda',
    }),
]

In [12]:
# 1. 레이블 인코딩 (이미 이전 단계에서 수행했다면 생략 가능)
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_20)
y_val_encoded = le.transform(y_val_20)

# 2. 파이프라인 및 모델 정의
# CNNFeatureGenerator는 재사용하거나 새로 생성할 수 있습니다. 여기서는 새로 생성합니다.
cnn_pipeline_xgb = CNNFeatureGenerator(
    num_classes=len(CFG.all_gestures),
    feature_dim=64,
    epochs=10,
    batch_size=128,
    lr=1e-4,
    verbose=True
)

xgb_model_cnn = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=8,
    eval_metric='mlogloss',
    random_state=CFG.seed,
    device='cuda' if torch.cuda.is_available() else 'cpu',
)

# 3. CNN 피처 생성 및 변환
print("CNN 피처 생성기 학습...")
cnn_pipeline_xgb.fit(X_train_20, y_train_encoded)

print("학습 및 검증 데이터에서 피처 추출 중...")
X_train_cnn_xgb = cnn_pipeline_xgb.transform(X_train_20)
X_val_cnn_xgb = cnn_pipeline_xgb.transform(X_val_20)

# 4. XGBoost 모델 학습
print("XGBoost 모델 학습 중...")
xgb_model_cnn.fit(X_train_cnn_xgb.to_pandas(), y_train_encoded)

# 5. 예측 및 평가
print("예측 및 평가 중...")
y_pred_encoded = xgb_model_cnn.predict(X_val_cnn_xgb.to_pandas())
y_pred = le.inverse_transform(y_pred_encoded)
score = calculate_contest_metric(y_val_20.to_numpy(), y_pred, CFG.target_gestures)

print(f"테스트 4 최종 점수: {score['final_score']:.4f}")
print(f"(Binary F1: {score['binary_f1']:.4f}, Macro F1: {score['macro_f1']:.4f})")

CNN 피처 생성기 학습...
CNN Feature Generator: Starting training on cuda for 10 epochs...
  Epoch 1/10, Loss: 2.353439
  Epoch 2/10, Loss: 2.138539
  Epoch 3/10, Loss: 2.056614
  Epoch 4/10, Loss: 2.004709
  Epoch 5/10, Loss: 1.968892
  Epoch 6/10, Loss: 1.940922
  Epoch 7/10, Loss: 1.918660
  Epoch 8/10, Loss: 1.899386
  Epoch 9/10, Loss: 1.883201
  Epoch 10/10, Loss: 1.869609
학습 및 검증 데이터에서 피처 추출 중...
CNN Feature Generator: Extracting features...
CNN Feature Generator: Extracting features...
XGBoost 모델 학습 중...
예측 및 평가 중...


c:\Users\namwonjin\Documents\Kaggle\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [16:55:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


테스트 4 최종 점수: 0.8193
(Binary F1: 0.8967, Macro F1: 0.7420)


In [14]:
ensemble = HybridFullStackingClassifier(
    pipelines=pipelines,
    model_defs=model_defs,
    n_splits = CFG.folds, 
    random_state = CFG.seed,
    meta_model=LogisticRegression(),
    n_bins=10,
    verbose=True
)

# Polars 데이터프레임을 입력으로 학습 시작
ensemble.fit(train_X_full, train_y_full)


Training Pipeline 1 + Model CatBoostClassifier
  Fold 1/5...
  Fold 2/5...
  Fold 3/5...
  Fold 4/5...
  Fold 5/5...

Training Pipeline 1 + Model XGBClassifier
  Fold 1/5...
  Fold 2/5...
  Fold 3/5...
  Fold 4/5...
  Fold 5/5...

Training Pipeline 2 + Model CatBoostClassifier
  Fold 1/5...
CNN Feature Generator: Starting training on cuda for 50 epochs...
  Epoch 1/50, Loss: 2.784440
  Epoch 2/50, Loss: 2.751245
  Epoch 3/50, Loss: 2.736000
  Epoch 4/50, Loss: 2.721005
  Epoch 5/50, Loss: 2.707359
  Epoch 6/50, Loss: 2.694403
  Epoch 7/50, Loss: 2.682366
  Epoch 8/50, Loss: 2.671793
  Epoch 9/50, Loss: 2.661504
  Epoch 10/50, Loss: 2.652891
  Epoch 11/50, Loss: 2.644945
  Epoch 12/50, Loss: 2.636849
  Epoch 13/50, Loss: 2.630491
  Epoch 14/50, Loss: 2.624812
  Epoch 15/50, Loss: 2.617692
  Epoch 16/50, Loss: 2.612284
  Epoch 17/50, Loss: 2.607305
  Epoch 18/50, Loss: 2.602419
  Epoch 19/50, Loss: 2.597887
  Epoch 20/50, Loss: 2.593732
  Epoch 21/50, Loss: 2.590371
  Epoch 22/50, Loss:

c:\Users\namwonjin\Documents\Kaggle\venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


HybridFullStackingClassifier(meta_model=LogisticRegression(),
                             model_defs=[(<class 'catboost.core.CatBoostClassifier'>,
                                          {'bootstrap_type': 'Poisson',
                                           'depth': 8,
                                           'early_stopping_rounds': 200,
                                           'eval_metric': 'MultiClass',
                                           'grow_policy': 'Depthwise',
                                           'iterations': 2000, 'l2_leaf_reg': 3,
                                           'learning_rate': 0.05,
                                           'loss_function': 'MultiClass',
                                           'random_seed': 42, 'subsample': 0....
                                                                                 'tof_1_v10',
                                                                                 'tof_1_v11',
                                                                                 'tof_1_v12',
                                                                                 'tof_1_v13',
                                                                                 'tof_1_v14',
                                                                                 'tof_1_v15',
                                                                                 'tof_1_v16',
                                                                                 'tof_1_v17',
                                                                                 'tof_1_v18',
                                                                                 'tof_1_v19',
                                                                                 'tof_1_v20',
                                                                                 'tof_1_v21',
                                                                                 'tof_1_v22',
                                                                                 'tof_1_v23',
                                                                                 'tof_1_v24',
                                                                                 'tof_1_v25',
                                                                                 'tof_1_v26',
                                                                                 'tof_1_v27',
                                                                                 'tof_1_v28',
                                                                                 'tof_1_v29', ...]))]),
                                        Pipeline(steps=[('cnn_features',
                                                         CNNFeatureGenerator(batch_size=128,
                                                                             epochs=50,
                                                                             feature_dim=64,
                                                                             num_classes=18))])])

In [15]:
test_X = test.drop(['row_id'])
predictions = ensemble.predict(test_X)

CNN Feature Generator: Extracting features...
CNN Feature Generator: Extracting features...
CNN Feature Generator: Extracting features...
CNN Feature Generator: Extracting features...
CNN Feature Generator: Extracting features...
CNN Feature Generator: Extracting features...
CNN Feature Generator: Extracting features...
CNN Feature Generator: Extracting features...
CNN Feature Generator: Extracting features...
CNN Feature Generator: Extracting features...


In [16]:
submission_df = pl.DataFrame({
    'row_id': test['row_id'],
    'gesture': predictions
})

# 4. CSV 파일로 저장
submission_df.write_csv('submission.csv')

print("\n'submission.csv' 파일 생성이 완료되었습니다!")
print(submission_df.head())


'submission.csv' 파일 생성이 완료되었습니다!
shape: (5, 2)
┌───────────────────┬───────────────┐
│ row_id            ┆ gesture       │
│ ---               ┆ ---           │
│ str               ┆ str           │
╞═══════════════════╪═══════════════╡
│ SEQ_000001_000000 ┆ Text on phone │
│ SEQ_000001_000001 ┆ Text on phone │
│ SEQ_000001_000002 ┆ Text on phone │
│ SEQ_000001_000003 ┆ Text on phone │
│ SEQ_000001_000004 ┆ Text on phone │
└───────────────────┴───────────────┘
